# Data scientists have their heads in the clouds.

One of the main concerns one developing neural networks is they can be quite computational intensive. Large networks over big data sets can take from hours to weeks of training time when running over CPU's or lead to memory errors.

Fortunately, the type of mathematical operations running behing your neural network are mainly matrix and vector multiplications (now Tensor - Flow makes more sense, right?). It happens that this tasks are well suited for the parallel architecture of GPUs. Take for example 3D computer graphics. There are a lot of such operations, like computation of lighting effects from normal maps, 3D effects, etc. GPUs were designed to handle such vector and matrix operations in parallel unlike a single core CPU that would handle matrix operations in serial form processing one element at a time [1].

In this session we will learn how to:

1. Sign up for an aws account
2. How to launch and connect to a cloud computer (*instance* in aws jargon) ready to run keras + tensorflow with GPU support
3. How to set up a jupyter notebook and access it from our favorite browser and build our own firs simple neural network

Last but not least, note that aws are not for free and are billed by hour. Remember to close your instance once when you are done or you may suffer a heart atack next time you receive an aws email. We will learn how to set up billing alarms so don't panic!

Ben always advises me to not perform live demos but, meh, let's do this together!

## 1. Signing up

Just go to [aws](aws.amazon.com) and follow the instructions to set up a new free account.

## 2. Create and connect to an instance

###  How to launch a new instance

Now that we have an aws account and are logged in, let's launch our elastic cloud computing (EC2) instance.

1. **Type "EC2" in the search bar and click on it** <img src="img/1-ec2.png" alt="EC2" style="width: 800px;"/>

2. **Click on Launch Instance** <img src="img/2-launchinstance.png" alt="EC2" style="width: 800px;"/>

3. **Go to AWS marketplace and search for deep learning ubuntu and click select** <img src="img/3-ami.png" alt="EC2" style="width: 800px;"/>

4. **Select the t2 instance. It is free so we do not get charged while we set up all our system. Once it is all set and ready, we can change the instance type to one with more memory and GPUs** <img src="img/4-instance.png" alt="EC2" style="width: 800px;"/>

5. **There are many options but by now let's restrict to the important: allow external connections so we can set up our jupyter notebook. Go to the *Configure Security Group* and click to *Add Rule*. Add the 8888 port and the 0.0.0.0/0 ip in the fields.** <img src="img/5-security.png" alt="EC2" style="width: 800px;"/>

6. **At this point we will be prompted in order to get the private key in a .pem file. We need to rename it and download it in a place where it is accessible. It will allow us to connect to our virtual machine via ssh. **<img src="img/6-pem.png" alt="EC2" style="width: 800px;"/>

7. **If everything is OK, we should get a message saying that our instance is launching. At this point, a massage would be great also.** <img src="img/7-success.png" alt="EC2" style="width: 800px;"/>

8. **Go to your istance manager. You should be able to see your instance in the list with its status. Note the public IP. This is what we will use to connect to it.** <img src="img/8-ip.png" alt="EC2" style="width: 800px;"/>

### Connect to the instance

Thumbs up! So far, you have your instance alive and kickin! Let's see how we can connect to it. Remember that you will need to access the .pem file we have just downloaded.

1. Open the terminal and go to the folder where you keep your .pem file. Before connecting, we need to change its settings so only the root user (you) can see it. To do this, type:
```sh
chmod 400 s2ds-nn.pem
```
2. Now you just need to type ```ssh -i s2ds-nn.pem ubuntu@34.248.241.207``` (remember to use the name of your .pem file and the current ip of your instance). Type yes when prompted and you should see:
<img src="img/9-chmodpem.png" alt="EC2" style="width: 200;"/>

** CONGRATULATIONS! You are now connected to your instance.**

## Setting up our jupyter environment

To this point we have achieved to set up and connect to a cloud instance. However, note that unless you are an expert usint terminal editors its quite difficult to test or play with neural networks. It could be useful though if we already have an script and we just want to run it, but in such case instead of an EC2, a EMR would suit better our needs. That's another story for another day (I totally should ask for amazon sponsorship at this point).

So, as we don't want to fight against vim (sorry!) let's set up our secured cloud jupyter notebook! Note that all we need is already preinstalled in the instance we launched.

1. **Create a go to a directory where we will store our keys. Then copy/paste the openssl command below and answer the questions.**
```sh
mkdir certs
cd certs
sudo openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mycert.pem -out mycert.pem
```
2. **Create an encrypted password to secure your notebook. Type ipython in the bash and, once in ipython type the next snippet. You will get a sha1 password that we will in a minute:**
```python
from IPython.lib import passwd
passwd("mysecretpassword")
exit
```
3. **Generate and edit jupyter configuration file by typing in the terminal:**
```sh
jupyter notebook --generate-config
```

4. **Open the generated config file using vi and copy the next snippet. You should  change the sha1 string by the one obtaine in step 2:**
Term:
```sh
vi ~/.jupyter/jupyter_notebook_config.py
```
Press "i" and then paste at top of the config file:
```python
c = get_config()  # get the config object
c.NotebookApp.certfile = u'/home/ubuntu/certs/mycert.pem' # path to the certificate we generated
c.NotebookApp.keyfile = u'/home/ubuntu/certs/mycert.pem' # path to the certificate key we generated
c.IPKernelApp.pylab = 'inline'  # in-line figure when using Matplotlib
c.NotebookApp.ip = '*'  # serve the notebooks locally
c.NotebookApp.open_browser = False  # do not open a browser window by default when using notebooks
c.NotebookApp.password = 'sha1:c7817706e1af:9e7495afcd18a12c96513f42423a0b6be257579a'  # this is the password hash that we generated earlier.
```
Press "ESC" + ":" + "wq" and press enter. You will save the modifications and exit the editor.

All right! Now we could already run our jupyter notebook in the terminal and display it in or browser. Nevertheless, if we close our current terminal, we would kill the process and our notebook would disapear. If we want to avoid to close our process, it is a good tip to run the jupyter notebook using tmux.

Just, type ```tmux``` in the terminal and a new terminal will appear. Just type in there ```jupyter notebook``` and detach from this window by "CTRL"+"b"+"d".

Let's check that everything is running correctly.

In your browser go to the pubic ip of your instance followed by :8888. Like:

https://34.248.241.207:8888

Accept the certificate and the jupyter notebook will appear! Just type your password and...that's it! Ready to go!

<img src="img/10-jupyternotebook.png" alt="EC2" style="width: 800px;"/>


## Final words

This is it, we have now or own cloud environment with all we need to quickly develop machine learning models and deep neural networks. One final advise to take into account is that as aws is a pay-per-use service it is recommended to code and test everything is working finely using the free insances. Then, once we want to train the model stop the instance, change it to a powerful one, and finally resume the instance and run the code.

Have fun!!

Juan

**Disclaimer**

I am not getting paid for talking about aws services, so choose whatever cloud services you prefer.

**Biblio**

1. [Quora thread on GPUS for neural network training](https://www.quora.com/Why-are-GPUs-well-suited-to-deep-learning)
2. [jupyter notebooks on aws](https://blog.keras.io/running-jupyter-notebooks-on-gpu-on-aws-a-starter-guide.html)